In [ ]:
#!/usr/bin/env python

import pandas as pd
import networkx as nx
from networkx.algorithms.bipartite.matrix import biadjacency_matrix
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
import random
from sklearn import metrics
import time
from joblib import Parallel, delayed
from sklearn.utils import parallel_backend
from itertools import product
from sklearn.metrics import roc_auc_score
from sklearn import svm
random.seed(1949) 
np.random.seed(1949) 

This file is for the study of ADRs prediction using SVM linear kernel. After the function in section 1 run, nested CV, CV and result of test set in section 2 can be ran separately.

## 1. Define the functions used for SVM-linear

In [ ]:
def option(str):
    global methodOption
    methodOption = str

### Functions for SVM-linear

In [ ]:
def option2(tf):
    global normalizationOption
    normalizationOption = tf

In [ ]:
def SVMloop(idx, X, X_new, y, y_new, Y_new, idx_test,c,g):
    # print(idx)
    # t = time.time()
    y_i = np.array(y[idx, :].copy()).tolist()
    if len(np.unique(y_i)) == 1:
        Y_new[idx, idx_test] = np.unique(y_i)
    else:

        if (methodOption == "SVMRBF1") | (methodOption == "SVMRBF2") |  (methodOption == "SVMRBF1&2"):
            svr = svm.SVR(kernel="rbf", C = c, gamma = g).fit(X, y_i)

        elif (methodOption == "SVML1") | (methodOption == "SVML2") |  (methodOption == "SVML1&2"):
            svr = svm.SVR(kernel="linear", C = c).fit(X, y_i)

        p_labels = svr.predict(X_new)
        Y_new[idx, idx_test] = p_labels

In [ ]:
def SVMforAllSe(matrix, feature_matrix1, feature_matrix2, idx_train, idx_test, c, g):
    if (methodOption == "SVMRBF1") | (methodOption == "SVML1"):
        feature_matrix = feature_matrix1.copy()
    elif (methodOption == "SVMRBF2") | (methodOption == "SVML2"):
        feature_matrix = feature_matrix2.copy()
    elif (methodOption == "SVMRBF1&2") | (methodOption == "SVML1&2"):
        feature_matrix = np.array(pd.concat([pd.DataFrame(feature_matrix1), pd.DataFrame(feature_matrix2)], axis = 1))

    if normalizationOption == True:
        feature_matrix = (feature_matrix.copy() - feature_matrix.mean()) / feature_matrix.std()
    X = np.array(feature_matrix[idx_train, :].copy()).tolist()
    X_new = np.array(feature_matrix[idx_test, :].copy()).tolist()
    y = matrix[:, idx_train].copy()
    y_new = matrix[:, idx_test].copy()
    global Y_new
    Y_new = matrix.copy().astype(float)

    m,n = matrix.shape 
    
    with parallel_backend('threading'):
        Parallel(n_jobs=5000)(delayed(SVMloop)(idx = i , X=X, X_new=X_new, y=y, y_new=y_new, Y_new=Y_new, \
            idx_test=idx_test,c=c,g=g) for i in range(m))
    
    
    return Y_new

### Function for generating features for common drugs

In [ ]:
def FeaturePreprocess(df_all, drug_nodes):
    
    drug_nodes_df = np.intersect1d(df_all.index, drug_nodes)
    df = df_all.loc[drug_nodes_df]
    _, q = df.shape
    drug_nodes_diff = np.setdiff1d(drug_nodes, (df.index).tolist())
    n = len(drug_nodes_diff)
    df_diff = pd.DataFrame(np.zeros(n*q).reshape(n,q))
    df_diff.index = drug_nodes_diff
    df_diff.columns = df.columns
    df_all = pd.concat([df, df_diff], axis = 0)
    featureMat = df_all.loc[drug_nodes]
    return np.array(featureMat)

### Function for cross validation and nested cross validation

In [ ]:
def fold(IDX1,IDX2,feature_matrix1,feature_matrix2,matrix,c,g):
    # IDX1 target index, need to be evaluated
    # IDX2 test index, masked
    target_idx = IDX1
    mask_idx = IDX2
    Ground_Truth = matrix.copy()
    side_effects_drug_relation_copy = matrix.copy()

    # target_idx = IDX2
    ### making all the links to predict as 0 ###############    
    for i in range(len(mask_idx)):
        side_effects_drug_relation_copy[:, mask_idx[i]] = 0
    
    m,n = side_effects_drug_relation_copy.shape

    drug_idx = list(range(n))
    existing_drug_idx = np.setdiff1d(drug_idx, mask_idx)
    
    print(methodOption + ' starts:')
    side_effects_drug_relation_fact = SVMforAllSe(matrix=matrix, feature_matrix1=feature_matrix1, feature_matrix2=feature_matrix2, idx_train=existing_drug_idx, idx_test=target_idx, c=c, g=g)
    # sys.stdout = real_stdout
    print(methodOption + ' ends:')


    # Set the out put of GNMF as prediction score
    score = side_effects_drug_relation_fact.copy()
    # score = score_mean.copy()

    
    pr_auc_all = 0
    roc_auc_all = 0
    

    print("proportion of ground truth:", sum(Ground_Truth[:, target_idx].ravel())/(Ground_Truth[:, target_idx].shape[0]*Ground_Truth[:, target_idx].shape[1]))

    print('---evaluation---')
    prec, recall, threshold = precision_recall_curve(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())
    pr_auc_all = auc(recall, prec)
    roc_auc_all = roc_auc_score(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())



    print("-----")

    print("AUC-PR all:", pr_auc_all)

    print("-----")

    print("AUC-ROC all:", roc_auc_all)

    print("-----")

    return pr_auc_all, roc_auc_all

In [ ]:
def innerfold(IDX1,IDX2,feature_matrix1,feature_matrix2,matrix,c,g):
    # IDX1 target index, need to be evaluated
    # IDX2 test index, masked

    target_idx = IDX1
    mask_idx = IDX2
    Ground_Truth = matrix.copy()
    side_effects_drug_relation_copy = matrix.copy()

    # target_idx = IDX2
    ### making all the links to predict as 0 ###############    
    for i in range(len(mask_idx)):
        side_effects_drug_relation_copy[:, mask_idx[i]] = 0
    
    m,n = side_effects_drug_relation_copy.shape

    drug_idx = list(range(n))
    existing_drug_idx = np.setdiff1d(drug_idx, mask_idx)
    
    # calculate the mean for each drug
    mean_side_effect_score = (Ground_Truth.copy()[:, existing_drug_idx]).mean(axis=1)
    score_mean = side_effects_drug_relation_copy.copy().astype(float)

    # Set the prediction into mean
    for i in range(m):
        score_mean[i, mask_idx] =  mean_side_effect_score[i]


    side_effects_drug_relation_fact = SVMforAllSe(matrix=matrix, feature_matrix1=feature_matrix1, feature_matrix2=feature_matrix2, idx_train=existing_drug_idx, idx_test=target_idx, c=c, g=g)


    # Set the out put of GNMF as prediction score
    score = side_effects_drug_relation_fact.copy()

    # Random score
    random_score = np.random.rand(m,n)
    

    pr_auc_all = 0
    roc_auc_all = 0
    

    prec, recall, threshold = precision_recall_curve(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())
    pr_auc_all = auc(recall, prec)
    roc_auc_all = roc_auc_score(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())


    return pr_auc_all, roc_auc_all

In [ ]:
def plotfold(IDX1,IDX2,feature_matrix1,feature_matrix2,matrix,c,g):
    # IDX1 target index, need to be evaluated
    # IDX2 test index, masked

    target_idx = IDX1
    mask_idx = IDX2
    Ground_Truth = matrix.copy()
    side_effects_drug_relation_copy = matrix.copy()
 
    for i in range(len(mask_idx)):
        side_effects_drug_relation_copy[:, mask_idx[i]] = 0
    
    m,n = side_effects_drug_relation_copy.shape

    drug_idx = list(range(n))
    existing_drug_idx = np.setdiff1d(drug_idx, mask_idx)
    
    # calculate the mean for each drug
    mean_side_effect_score = (Ground_Truth.copy()[:, existing_drug_idx]).mean(axis=1)
    score_mean = side_effects_drug_relation_copy.copy().astype(float)

    # Set the prediction into mean
    for i in range(m):
        score_mean[i, mask_idx] =  mean_side_effect_score[i]

    print(methodOption + ' starts:')
    side_effects_drug_relation_fact = SVMforAllSe(matrix=matrix, feature_matrix1=feature_matrix1, feature_matrix2=feature_matrix2, idx_train=existing_drug_idx, idx_test=target_idx, c=c, g=g)
    # sys.stdout = real_stdout
    print(methodOption + ' ends:')


    # Set the out put of GNMF as prediction score
    score = side_effects_drug_relation_fact.copy()
    # score = score_mean.copy()

    
    pr_auc_all = 0
    roc_auc_all = 0
    

    print("proportion of ground truth:", sum(Ground_Truth[:, target_idx].ravel())/(Ground_Truth[:, target_idx].shape[0]*Ground_Truth[:, target_idx].shape[1]))

    print('---evaluation---')

    prec, recall, prthreshold = precision_recall_curve(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())
    pr_auc_all = auc(recall, prec)
    
    fpr, tpr, rocthreshold = metrics.roc_curve(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())
    roc_auc_all = auc(fpr, tpr)

    print("-----")

    print("AUC-PR all:", pr_auc_all)

    print("-----")

    print("AUC-ROC all:", roc_auc_all)

    print("-----")

    Out1 = pd.DataFrame([prec, recall, prthreshold])
    Out2 = pd.DataFrame([fpr, tpr, rocthreshold])
    return Out1, Out2


### Functions for assigning arguments of CV and nested CV, as well as finding the best hyperparameters

In [ ]:
def setvar_tune(size):
# set var for hyper pars tuning size is the hyper par size ALL_...

    global ALL_AUCPR_all
    global ALL_AUROC_all

    ALL_AUCPR_all = np.zeros(size)
    ALL_AUROC_all = np.zeros(size)

In [ ]:
def setvar_cv(FOLDS):
# set var for cv 

    global AUC_roc_all
    global AUC_pr_all

    AUC_roc_all = np.zeros(FOLDS)
    AUC_pr_all = np.zeros(FOLDS)


In [ ]:
def asgvar_tune(idx, results):
    # assign var for cv from results
    # f: size of hyper pars

    ALL_AUCPR_all[idx] = results[0]
    ALL_AUROC_all[idx] = results[1]


In [ ]:
def asgvar_cv(f, results):
    # assign var for cv from results
    # f: size of hyper pars

    AUC_pr_all[f] = results[0]
    AUC_roc_all[f] = results[1]

In [ ]:
def tuning_results(tuneVar):
    idx = np.argmax(ALL_AUCPR_all)
    Var = tuneVar[idx]
    Value = ALL_AUCPR_all[idx]

    print("best hyperpar: ", Var)
    print("AUPRC per drug: ", Value)

    return Var, Value

In [ ]:
def setvar_besttune(innerfolds):
    global besttunevalue
    global besttunevar
    besttunevalue = np.zeros(innerfolds) # best metric value
    besttunevar = np.zeros(innerfolds) # the value of best var
    besttunevar = besttunevar.tolist()

In [ ]:
def asg_besttune(f, value, var):
    besttunevalue[f] = value
    besttunevar[f] = var

In [ ]:
def besttune():
    idx = np.argmax(besttunevalue)
    value = besttunevalue[idx]
    var = besttunevar[idx]
    return value, var

In [ ]:
def cv_results():

    print("Mean AUC_pr_all", AUC_pr_all.mean()," ", "Standard Deviation:", AUC_pr_all.std())
    print("Mean AUC_roc_all", AUC_roc_all.mean()," ", "Standard Deviation:", AUC_roc_all.std())
    print("-----------")
    results = np.array([AUC_pr_all, AUC_roc_all])

    return results

### Function for parallel computation

In [ ]:
def tuning_loop(innermatrix, idx_train_inner, idx_test_inner, feature_matrix_inner1, feature_matrix_inner2, hyperparList, i):
    
    c,g = hyperparList[i]
    idx_target_inner = idx_test_inner
    results = innerfold(idx_target_inner,idx_test_inner,feature_matrix1=feature_matrix_inner1,feature_matrix2=feature_matrix_inner2,matrix=innermatrix,c=c,g=g)
    asgvar_tune(i, results=results)

### Main function

In [ ]:
def main(method_option,normalization=True,Validation="nested_cv",sets="intersect", c=0, g=0):

    random.seed(1949) # for dataset split
    np.random.seed(1949) # for matrix initialization
    option(method_option)
    option2(normalization)

    random.seed(1949) # for dataset split
    np.random.seed(1949) # for matrix initialization
    df = pd.read_csv("data/side-effect-and-drug_name_upper.tsv",sep = "\t")
    drug_id = df["drugbank_id"] # put col of df in var
    drug_name = df["drugbank_name"]
    side_effect = df["side_effect_name"]
    
    
    edgelist1 = zip(side_effect, drug_name)
    ##making Biparite Graph##
    B = nx.DiGraph()
    B.add_nodes_from(side_effect,bipartite = 0)
    B.add_nodes_from(drug_name,bipartite = 1)
    B.add_edges_from(edgelist1)
    # B.add_weighted_edges_from(edgelist2)
    drug_nodes = {n for n, d in B.nodes(data=True) if d['bipartite']==1}
    side_effect_nodes = {n for n, d in B.nodes(data=True) if d['bipartite']==0}
    drug_nodes = list(drug_nodes)
    drug_nodes.sort()
    side_effect_nodes = list(side_effect_nodes)
    side_effect_nodes.sort()
    ###Getting the Bi-Adjacency matrix between side effects and drugs ###################
    matrix_all = biadjacency_matrix(B, row_order = side_effect_nodes, column_order = drug_nodes) # create biadjacency matrix for drug side effect graph
    matrix_all = matrix_all.A
    m_all,n_all = matrix_all.shape # number of side effect # number of drug
    
    
    ### Setting validation set / training set / testing set ###
    validate_sz = int(0.25 * n_all)
    IDX_all = list(range(n_all))
    random.shuffle(IDX_all)
    IDX_validate = sorted(IDX_all[0:validate_sz])
    print("first few validation set idx:")
    print(IDX_validate[0:10])
    IDX_validate_diff = np.setdiff1d(IDX_all, IDX_validate)
    matrix = matrix_all[:, IDX_validate_diff].copy()
    
    df1 = pd.read_csv("data/intersection_DGIdb_mat.tsv",sep = "\t")
    df2 = pd.read_csv("data/intersection_Fingerprint_mat.tsv",sep = "\t")
    featureMat1_all = FeaturePreprocess(df1, drug_nodes=drug_nodes)
    featureMat2_all = FeaturePreprocess(df2, drug_nodes=drug_nodes)
    featureMat1 = featureMat1_all[IDX_validate_diff, :].copy()
    featureMat2 = featureMat2_all[IDX_validate_diff, :].copy()
    
    
    non_zero_idx_union = np.hstack(np.where(~((featureMat1.sum(1) == 0) & (featureMat2.sum(1) == 0))))
    non_zero_idx_missing = np.hstack(np.where(~(~(featureMat1.sum(1) == 0) & ~(featureMat2.sum(1) == 0))))
    non_zero_idx_intersect = np.hstack(np.where(~(featureMat1.sum(1) == 0) & ~(featureMat2.sum(1) == 0)))
    if sets == "union":
        # union
        matrix = matrix[:, non_zero_idx_union].copy()
        featureMat1 = featureMat1[non_zero_idx_union, :].copy()
        featureMat2 = featureMat2[non_zero_idx_union, :].copy()
    elif sets == "intersect":
        # intersect
        non_zero_idx_intersect_all = np.hstack(np.where(~(featureMat1_all.sum(1) == 0) & ~(featureMat2_all.sum(1) == 0)))
    
        matrix_all = matrix_all[:, non_zero_idx_intersect_all].copy()
        featureMat1_all = featureMat1_all[non_zero_idx_intersect_all, :].copy()
        featureMat2_all = featureMat2_all[non_zero_idx_intersect_all, :].copy()
    
        matrix = matrix[:, non_zero_idx_intersect].copy()
        featureMat1 = featureMat1[non_zero_idx_intersect, :].copy()
        featureMat2 = featureMat2[non_zero_idx_intersect, :].copy()
    
        IDX_validate = np.setdiff1d(non_zero_idx_intersect_all, IDX_validate_diff)
        IDX_validate_diff = np.setdiff1d(non_zero_idx_intersect_all, IDX_validate)
    
        drug_nodes_intersect_all = np.array(drug_nodes)[non_zero_idx_intersect_all]
        drug_nodes_intersect_validate_diff = np.array(drug_nodes)[IDX_validate_diff]
        drug_nodes_intersect_validate = np.array(drug_nodes)[IDX_validate]
    
        IDX_validate = np.array([x for x in range(len(drug_nodes_intersect_all)) if drug_nodes_intersect_all[x] in drug_nodes_intersect_validate])
        IDX_validate_diff = np.array([x for x in range(len(drug_nodes_intersect_all)) if drug_nodes_intersect_all[x] in drug_nodes_intersect_validate_diff])
    

    m,n = matrix.shape # number of side effect # number of drug


    random.seed(1949) # for dataset split
    np.random.seed(1949) # for matrix initialization



    FOLDS = 5
    innerFOLDS = 4
    ####for test sets####
    setvar_cv(FOLDS)

    sz = n
    IDX = list(range(sz))
    fsz = int(sz/FOLDS)
    random.shuffle(IDX)
    IDX = np.array(IDX)
    offset = 0

    innersz = sz - fsz
    innerIDX = list(range(innersz))
    random.shuffle(innerIDX)
    innerIDX = np.array(innerIDX)
    innerfsz = int(innersz / innerFOLDS)
    inneroffset = 0
    # setvar_cv(FOLDS=FOLDS)

    if Validation == "nested_cv":
        print("---------- nested cv start ----------")
        for f in range(FOLDS):  # range(FOLDS):
            offset = 0 + fsz*f 
            idx_test = IDX[offset:offset + fsz]
    
            idx_train = IDX[np.setdiff1d(np.arange(len(IDX)), np.arange(offset,offset + fsz))]
            print("Fold:",f)
                
            innermatrix = matrix[:, idx_train]
    
            innerfeatureMat1 = featureMat1[idx_train, :].copy()
            innerfeatureMat2 = featureMat2[idx_train, :].copy()
    
            setvar_besttune(innerFOLDS)
    
            for innerf in range(innerFOLDS):
                inneroffset = 0 + innerf*innerfsz
                idx_test_inner = innerIDX[inneroffset:inneroffset + innerfsz]
                idx_train_inner = innerIDX[np.array(np.setdiff1d(np.arange(len(idx_train)), np.arange(inneroffset,inneroffset + innerfsz)))]
    
                print("Inner Fold:", innerf)

                if (methodOption == "SVMRBF1") | (methodOption == "SVMRBF2") |  (methodOption == "SVMRBF1&2"):
                    gamma = (10**np.arange(-4, -1, 1, dtype=float)).tolist()
                elif (methodOption == "SVML1") | (methodOption == "SVML2") |  (methodOption == "SVML1&2"):
                    gamma = (10**np.arange(1, 2, 1, dtype=float)).tolist()
                C = (10**np.arange(-3, -2, 1, dtype=float)).tolist()
                hyperparList = list(product(C, gamma))
    
                setvar_tune(len(hyperparList))
    
                with parallel_backend('threading'):
                    Parallel(n_jobs=1)(delayed(tuning_loop)(innermatrix = innermatrix, idx_train_inner = idx_train_inner, \
                            idx_test_inner = idx_test_inner, feature_matrix_inner1 = innerfeatureMat1, feature_matrix_inner2 = innerfeatureMat2, \
                                hyperparList = hyperparList, i = i) \
                                    for i in range(len(hyperparList)))
    
                # tuning_plot(tuneVar=C, tune="C")
                hyperpars, evalValue = tuning_results(tuneVar=hyperparList)
    
    
    
                asg_besttune(innerf, value=evalValue, var=hyperpars)
                    
            _, bestHyperPars = besttune()
    
            print("--- tuning end ---")
            c, g = bestHyperPars
            idx_target = idx_test
            print('target size:', len(idx_target))
    
            print("------ C: ", c, "gamma: ", g, "------")
    
            results = fold(idx_target,idx_test,featureMat1,featureMat2,matrix,c=c,g=g)
            asgvar_cv(f=f, results=results)

        out = cv_results()
        return out

    elif Validation == "cv":
        print("---------- cv start ----------")
        setvar_besttune(FOLDS)

        for f in range(FOLDS):
            offset = 0 + fsz*f 
            idx_test = IDX[offset:offset + fsz]
            idx_train = IDX[np.setdiff1d(np.arange(len(IDX)), np.arange(offset,offset + fsz))]

            print("Fold:", f)

            if methodOption == "SVMRBF1" | methodOption == "SVMRBF2" |  methodOption == "SVMRBF1&2":
                gamma = (10**np.arange(-4, -1, 1, dtype=float)).tolist()
            elif methodOption == "SVML1" | methodOption == "SVML2" |  methodOption == "SVML1&2":
                gamma = (10**np.arange(1, 2, 1, dtype=float)).tolist()
            C = (10**np.arange(-1, -2, 1, dtype=float)).tolist()
            hyperparList = list(product(C, gamma))

            setvar_tune(len(hyperparList))
    
            with parallel_backend('threading'):
                Parallel(n_jobs=1)(delayed(tuning_loop)(innermatrix = matrix, idx_train_inner = idx_train, \
                        idx_test_inner = idx_test, feature_matrix_inner1 = featureMat1, feature_matrix_inner2 = featureMat2, \
                            hyperparList = hyperparList, i = i) \
                                    for i in range(len(hyperparList)))
    
            hyperpars, evalValue = tuning_results(tuneVar=hyperparList)
            asg_besttune(f, value=evalValue, var=hyperpars)

    
        print("--- tuning end ---")
        # cv_results()
        _, bestHyperPars = besttune()
    elif Validation == "Validation":

        # validation
        idx_test = IDX_validate
        idx_train = IDX_validate_diff
        idx_target = idx_test
        print('target size:', len(idx_target))
        print("------ C: ", c, "gamma: ", g, "------")
        results = fold(idx_target,idx_test,featureMat1_all,featureMat2_all,matrix_all,c=c,g=g)
        return
    elif Validation == "plot":
        
        # validation
        idx_test = IDX_validate
        idx_train = IDX_validate_diff
        idx_target = idx_test
        print('target size:', len(idx_target))
        print("------ C: ", c, "gamma: ", g, "------")
        pr, roc = plotfold(idx_target,idx_test,featureMat1_all,featureMat2_all,matrix_all,c=c,g=g)
        return pr, roc


## 2. Nested CV and CV for SVM-linear

### 2.1. Nested CV

Running the nested CV for feature DGI

In [ ]:
results_SVML_mean, results_SVML = main(method_option="SVML1", Validation="nested_cv") # C 0.001

Running the nested CV for feature Chem

In [ ]:
results_SVML_mean, results_SVML = main(method_option="SVML2", Validation="nested_cv") # C 0.001

Running the nested CV for feature DGI and Chem

In [ ]:
results_SVML_mean, results_SVML = main(method_option="SVML1&2", Validation="nested_cv") # C 0.001

### 2.2. CV to tune hyperparameters for independent test set

Running CV for DGI. The best hyperparameter is $C=0.001$.

In [ ]:
main(method_option="SVML1",Validation="cv") # C 0.001

Running CV for Chem. The best hyperparameter is $C=0.001$.

In [ ]:
main(method_option="SVML2",Validation="cv")

Running CV for DGI and Chem. The best hyperparameter is $C=0.001$.

In [ ]:
main(method_option="SVML1&2",Validation="cv")

### 2.3. Independent test set

DGI

In [ ]:
main(method_option="SVML1", Validation="Validation", c=0.001) 

Chem

In [ ]:
main(method_option="SVML2", Validation="Validation", c=0.001) 

Integration of DGI and Chem

In [ ]:
main(method_option="SVML1&2", Validation="Validation", c=0.001) 

### * Save data for PR and ROC

In [ ]:
SVML1_pr, SVML1_roc = \
    main(method_option="SVML1", Validation="plot", c=0.001)
SVML1_pr.T.to_csv("Figs/SVML1_pr.csv", index=False)
SVML1_roc.T.to_csv("Figs/SVML1_roc.csv", index=False)

In [ ]:
SVML2_pr, SVML2_roc = \
    main(method_option="SVML2", Validation="plot", c=0.001)
SVML2_pr.T.to_csv("Figs/SVML2_pr.csv", index=False)
SVML2_roc.T.to_csv("Figs/SVML2_roc.csv", index=False)

In [ ]:
SVML12_pr, SVML12_roc = \
    main(method_option="SVML1&2", Validation="plot", c=0.001)
SVML12_pr.T.to_csv("Figs/SVML12_pr.csv", index=False)
SVML12_roc.T.to_csv("Figs/SVML12_roc.csv", index=False)